<a href="https://colab.research.google.com/github/aviraltyagi/DeepLearningArchitectures/blob/main/AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Input, Layer, Dropout, Lambda, MaxPool2D
import typing

In [ ]:
class CustomConvLayer(Layer):
  def __init__(self, filter, kernelSize, stride, padding):
    super(CustomConvLayer, self).__init__()
    self.Conv = Conv2D(filters= filter, kernel_size= kernelSize, strides= stride, activation= 'relu', padding= padding)
    self.Pool = MaxPool2D(pool_size= (3,3), strides= 2, padding= 'valid')
    self.Lambda = Lambda(tf.nn.local_response_normalization)

  def call(self, input, training = False, isPool = True, isLambda = True):
    x = self.Conv(input, training = training)
    if(isPool):
      x = self.Pool(x)
    if(isLambda):
      x = self.Lambda(x)
    
    return x

In [ ]:
class CustomAlexNet(Model):
  def AlexNet(input_shape: typing.Tuple[int], classes: int) -> Model:
    x_input = Input(shape= input_shape)

    # Layer 1
    x = CustomConvLayer(filter= 96, kernelSize= (11,11), stride= (4,4), padding= 'valid')(x_input, training = True)

    # Layer 2
    x = CustomConvLayer(filter= 256, kernelSize= (5,5), stride= (1,1), padding= 'same')(x, training = True)

    # Layer 3
    x = CustomConvLayer(filter= 384, kernelSize= (3,3), stride= (1,1), padding='same')(x, training = True, isPool = False)

    # Layer 4
    x = CustomConvLayer(filter= 384, kernelSize= (3,3), stride= (1,1), padding='same')(x, training = True, isPool = False)

    # Layer 5
    x = CustomConvLayer(filter= 256, kernelSize= (3,3), stride= (1,1), padding='same')(x, training = True)

    # Layer 6
    x = Flatten()(x)
    x = Dense(units= 4096, activation= 'relu')(x)
    #x = Dropout(0.5)(x)

    # Layer 7
    x = Dense(units= 4096, activation= 'relu')(x)
    #x = Dropout(0.5)(x)

    # Layer 8
    output = Dense(units = classes, activation= 'softmax')(x)

    model = Model(inputs = x_input, outputs= output)

    return model

In [ ]:
model = CustomAlexNet.AlexNet(input_shape= (227, 227, 3), classes= 10)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
custom_conv_layer_5 (CustomC (None, 27, 27, 96)        34944     
_________________________________________________________________
custom_conv_layer_6 (CustomC (None, 13, 13, 256)       614656    
_________________________________________________________________
custom_conv_layer_7 (CustomC (None, 13, 13, 384)       885120    
_________________________________________________________________
custom_conv_layer_8 (CustomC (None, 13, 13, 384)       1327488   
_________________________________________________________________
custom_conv_layer_9 (CustomC (None, 6, 6, 256)         884992    
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0   